# Úkol č. 4 - regrese
**Deadline úkolu je uveden na [course pages](https://courses.fit.cvut.cz/BI-VZD/homeworks/index.html).**

  * Cílem tohoto úkolu je vyzkoušet si řešit regresní problém na reálných datech.
  
> **Nejdůležitější na úkolu je to, abyste udělali vše procesně správně: korektní rozdělení datasetu, ladění hyperparametrů, vyhodnocení výsledků atp.**

## Dataset

  * Zdrojem dat je soubor `LifeExpectancyData.csv` na course pages (originál zde: https://www.kaggle.com/kumarajarshi/life-expectancy-who).
  * Popis datasetu najdete na uvedené stránce s originálem datasetu.
  * Cílová (vysvětlovaná) proměnná se jmenuje `Life expectancy `.
  

## Pokyny k vypracování
Body zadání, za jejichž (poctivé) vypracování získáte 12 bodů:

  1. Odeberte z dat body u kterých neznáte vysvětlovanou proměnnou.
  1. Rozdělte data na trénovací a testovací množinu.
  1. Proveďte základní průzkum dat. Na jeho základě adekvátně reagujte na problematické věci v datech (chybějící hodnoty, atd.).
  1. Aplikujte lineární a hřebenovou regresi a výsledky řádně vyhodnoťte:
    * K měření chyby použijte `mean_absolute_error`.
    * Experimentujte s tvorbou nových příznaků (na základě těch dostupných).
    * Experimentujte se standardizací/normalizací dat.
    * Vyberte si hyperparametry modelů k ladění a najděte jejich nejlepší hodnoty.
  1. Použijte i jiný model než jen lineární a hřebenovou regresi.


## Poznámky k odevzdání

  * Řiďte se pokyny ze stránky https://courses.fit.cvut.cz/BI-VZD/homeworks/index.html.
  * Odevzdejte tento Jupyter Notebook.
  * Opravující Vám může umožnit úkol dodělat či opravit a získat tak další body. První verze je ale důležitá a bude-li odbytá, budete za to penalizováni.

In [1]:
import pandas as pd


In [2]:
data = pd.read_csv('./LifeExpectancyData.csv')
display(data)

,Country,Year,Status,Life expectancy,Adult Mortality,infant deaths,Alcohol,percentage expenditure,Hepatitis B,Measles,...,Polio,Total expenditure,Diphtheria,HIV/AIDS,GDP,Population,thinness 1-19 years,thinness 5-9 years,Income composition of resources,Schooling
0,Afghanistan,2015,Developing,65.0,263.0,62,0.01,71.279624,65.0,1154,...,6.0,8.16,65.0,0.1,584.259210,33736494.0,17.2,17.3,0.479,10.1
1,Afghanistan,2014,Developing,59.9,271.0,64,0.01,73.523582,62.0,492,...,58.0,8.18,62.0,0.1,612.696514,327582.0,17.5,17.5,0.476,10.0
2,Afghanistan,2013,Developing,59.9,268.0,66,0.01,73.219243,64.0,430,...,62.0,8.13,64.0,0.1,631.744976,31731688.0,17.7,17.7,0.470,9.9
3,Afghanistan,2012,Developing,59.5,272.0,69,0.01,78.184215,67.0,2787,...,67.0,8.52,67.0,0.1,669.959000,3696958.0,17.9,18.0,0.463,9.8
4,Afghanistan,2011,Developing,59.2,275.0,71,0.01,7.097109,68.0,3013,...,68.0,7.87,68.0,0.1,63.537231,2978599.0,18.2,18.2,0.454,9.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2933,Zimbabwe,2004,Developing,44.3,723.0,27,4.36,0.000000,68.0,31,...,67.0,7.13,65.0,33.6,454.366654,12777511.0,9.4,9.4,0.407,9.2
2934,Zimbabwe,2003,Developing,44.5,715.0,26,4.06,0.000000,7.0,998,...,7.0,6.52,68.0,36.7,453.351155,12633897.0,9.8,9.9,0.418,9.5
2935,Zimbabwe,2002,Developing,44.8,73.0,25,4.43,0.000000,73.0,304,...,73.0,6.53,71.0,39.8,57.348340,125525.0,1.2,1.3,0.427,10.0
2936,Zimbabwe,2001,Developing,45.3,686.0,25,1.72,0.000000,76.0,529,...,76.0,6.16,75.0,42.1,548.587312,12366165.0,1.6,1.7,0.427,9.8


In [13]:
data.isnull().sum()[data.isnull().sum()>0]

Life expectancy                     10
Adult Mortality                     10
Alcohol                            194
Hepatitis B                        553
 BMI                                34
Polio                               19
Total expenditure                  226
Diphtheria                          19
GDP                                448
Population                         652
 thinness  1-19 years               34
 thinness 5-9 years                 34
Income composition of resources    167
Schooling                          163
dtype: int64

Folowing cell output shows data doesn't really make sense. How on earth Georgia could have 43 people in 2008?

In [20]:
data[data['Population']<100][['Country','Year','Population']]

,Country,Year,Population
985,Georgia,2008,43.0
1603,Maldives,2014,41.0
1608,Maldives,2009,36.0
1614,Maldives,2003,34.0


In [28]:
data.describe()

,Year,Life expectancy,Adult Mortality,infant deaths,Alcohol,percentage expenditure,Hepatitis B,Measles,BMI,under-five deaths,Polio,Total expenditure,Diphtheria,HIV/AIDS,GDP,Population,thinness 1-19 years,thinness 5-9 years,Income composition of resources,Schooling
count,2938.000000,2928.000000,2928.000000,2938.000000,2744.000000,2938.000000,2385.000000,2938.000000,2904.000000,2938.000000,2919.000000,2712.00000,2919.000000,2938.000000,2490.000000,2.286000e+03,2904.000000,2904.000000,2771.000000,2775.000000
mean,2007.518720,69.224932,164.796448,30.303948,4.602861,738.251295,80.940461,2419.592240,38.321247,42.035739,82.550188,5.93819,82.324084,1.742103,7483.158469,1.275338e+07,4.839704,4.870317,0.627551,11.992793
std,4.613841,9.523867,124.292079,117.926501,4.052413,1987.914858,25.070016,11467.272489,20.044034,160.445548,23.428046,2.49832,23.716912,5.077785,14270.169342,6.101210e+07,4.420195,4.508882,0.210904,3.358920
min,2000.000000,36.300000,1.000000,0.000000,0.010000,0.000000,1.000000,0.000000,1.000000,0.000000,3.000000,0.37000,2.000000,0.100000,1.681350,3.400000e+01,0.100000,0.100000,0.000000,0.000000
25%,2004.000000,63.100000,74.000000,0.000000,0.877500,4.685343,77.000000,0.000000,19.300000,0.000000,78.000000,4.26000,78.000000,0.100000,463.935626,1.957932e+05,1.600000,1.500000,0.493000,10.100000
50%,2008.000000,72.100000,144.000000,3.000000,3.755000,64.912906,92.000000,17.000000,43.500000,4.000000,93.000000,5.75500,93.000000,0.100000,1766.947595,1.386542e+06,3.300000,3.300000,0.677000,12.300000
75%,2012.000000,75.700000,228.000000,22.000000,7.702500,441.534144,97.000000,360.250000,56.200000,28.000000,97.000000,7.49250,97.000000,0.800000,5910.806335,7.420359e+06,7.200000,7.200000,0.779000,14.300000
max,2015.000000,89.000000,723.000000,1800.000000,17.870000,19479.911610,99.000000,212183.000000,87.300000,2500.000000,99.000000,17.60000,99.000000,50.600000,119172.741800,1.293859e+09,27.700000,28.600000,0.948000,20.700000


Minimal GDP seemed unreasonable, so I decided to compare range of possible GDP per capita values with [wikipedia](https://en.wikipedia.org/wiki/List_of_countries_by_past_and_projected_GDP_(PPP)_per_capita#IMF_estimates_between_2000_and_2009)

In [112]:
gdp_web = pd.read_html('https://en.wikipedia.org/wiki/List_of_countries_by_past_and_projected_GDP_(PPP)_per_capita#IMF_estimates_between_2000_and_2009')
true_gdp = pd.DataFrame(data={'Country':gdp_web[0][gdp_web[0].columns[0]]});
for i in range(len(gdp_web) - 1):
    gdp_web[i].rename(columns = {'Country (or dependent territory)':'Country'}, inplace=True)
    true_gdp = true_gdp.join(other=gdp_web[i].set_index('Country'), on='Country', how='outer')
true_gdp = pd.concat([true_gdp['Country'],true_gdp.iloc[:,21:37]],axis=1).set_index('Country')
display(true_gdp)

,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
Country,,,,,,,,,,,,,,,,
Afghanistan,NaN,NaN,886.0,942.0,938.0,1045.0,1100.0,1233.0,1273.0,1507.0,1608.0,1694.0,1953.0,2010.0,2120.0,2136.0
Albania,4331.0,4838.0,5153.0,5559.0,6049.0,6616.0,7264.0,7965.0,8797.0,9223.0,9724.0,10208.0,10526.0,10571.0,11259.0,11662.0
Algeria,8605.0,8926.0,9435.0,10150.0,10710.0,11521.0,11890.0,12432.0,12698.0,12754.0,13105.0,13480.0,13264.0,13003.0,12940.0,11945.0
Angola,3136.0,3242.0,3634.0,3702.0,4095.0,4715.0,5261.0,5980.0,6580.0,6492.0,6686.0,6857.0,7624.0,7948.0,8508.0,7669.0
Antigua and Barbuda,17012.0,16247.0,16495.0,17629.0,18938.0,20558.0,23622.0,26219.0,26436.0,23162.0,21426.0,21224.0,20393.0,19499.0,19674.0,19456.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Venezuela,11652.0,12109.0,11029.0,10197.0,12191.0,13651.0,15215.0,16733.0,17689.0,16997.0,16691.0,17494.0,18562.0,18871.0,18215.0,17011.0
Vietnam,2492.0,2686.0,2884.0,3123.0,3426.0,3764.0,4110.0,4479.0,4778.0,5025.0,5356.0,5760.0,6329.0,6689.0,7203.0,7556.0
Yemen,2873.0,2954.0,3023.0,3096.0,3204.0,3381.0,3486.0,3588.0,3678.0,3734.0,3950.0,3417.0,3301.0,3479.0,3434.0,2423.0


In [113]:
true_gdp.describe()

,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
count,187.000000,188.000000,190.000000,191.000000,192.000000,192.000000,192.000000,193.000000,193.000000,193.000000,193.000000,193.000000,193.00000,193.000000,193.000000,193.000000
mean,11810.919786,12314.101064,12564.078947,13089.089005,13962.505208,14743.270833,15729.468750,16933.238342,17219.310881,16774.797927,17445.538860,18215.652850,18786.00000,19283.989637,19548.295337,19039.321244
std,15516.465819,15819.642318,16102.996856,16643.950522,17822.942082,18279.790230,19207.517827,20109.754935,19521.135194,19097.327716,20316.114831,21873.783468,22624.78675,22914.176524,22437.069489,20128.618729
min,438.000000,426.000000,419.000000,422.000000,434.000000,445.000000,468.000000,494.000000,518.000000,521.000000,547.000000,578.000000,581.00000,656.000000,720.000000,787.000000
25%,2435.000000,2433.750000,2536.500000,2530.500000,2690.750000,2963.750000,3033.750000,3347.000000,3424.000000,3543.000000,3696.000000,3417.000000,3654.00000,3935.000000,4237.000000,4330.000000
50%,6310.000000,6533.000000,6652.000000,6938.000000,7425.000000,7781.000000,8318.000000,9077.000000,9664.000000,9803.000000,10029.000000,10243.000000,10555.00000,11106.000000,11411.000000,11945.000000
75%,14169.500000,15733.000000,16413.750000,16803.500000,18149.000000,20103.500000,21914.250000,25750.000000,25037.000000,23162.000000,22520.000000,22862.000000,24646.00000,26045.000000,26754.000000,26810.000000
max,102898.000000,100721.000000,105668.000000,106752.000000,119789.000000,116805.000000,130807.000000,133290.000000,118421.000000,127921.000000,145583.000000,163740.000000,169698.00000,161194.000000,143222.000000,109014.000000


Nothing better comes to my head then completely replace original GDP values with data, downloaded from Wiki